# Egenverdier og evenvektorer (MIP 10.8)

In [1]:
import sympy as sp
import numpy as np

In [2]:
A = sp.Matrix([
    [0, .5, .5],
    [1, 0, 0],
    [0, .5, .5]
])

In [3]:
def finn_egenvektorer_og_egenverdier(A):
    assert A.shape[0] == A.shape[1], "matrisen A skal være kvadratisk"

    t = sp.symbols('t')  # Definerer symbolet t, som brukes i det karakteristiske polynomet
    B = A - t * sp.eye(A.shape[0])  # Lager matrisen B = A - t*I, hvor I er identitetsmatrisen
    karakteristisk_polynom = B.det()  # Finner determinant av B, som gir det karakteristiske polynomet
    
    egenverdier = sp.solve(karakteristisk_polynom)  # Løser det karakteristiske polynomet for å finne egenverdiene

    # Lager en liste med tupler av evenverdier, multiplisiteter og egenvektorer
    res = []
    # for ev in sorted(egenverdier, key=lambda x: -np.abs(x)):
    for ev in egenverdier:
        egenvektorer = (A - ev * sp.eye(A.shape[0])).nullspace()
        res.append((ev, len(egenvektorer), egenvektorer))
    
    return res  # Returnerer en liste med tupler av evenverdier, multiplisiteter og egenvektorer

In [4]:
def skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, egenvektorer):
    for key, _, val in sorted(egenvektorer):
        print('egenverdi:     ', key)
        print('egenvektor:    ', val[0])
        print('A @ evenvektor:', A @ val[0])
        print()

In [5]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

egenverdi:      -0.500000000000000
egenvektor:     Matrix([[1.00000000000000], [-2.00000000000000], [1]])
A @ evenvektor: Matrix([[-0.500000000000000], [1.00000000000000], [-0.500000000000000]])

egenverdi:      0.0
egenvektor:     Matrix([[0], [-1.00000000000000], [1]])
A @ evenvektor: Matrix([[0], [0], [0]])

egenverdi:      1.00000000000000
egenvektor:     Matrix([[1.00000000000000], [1.00000000000000], [1]])
A @ evenvektor: Matrix([[1.00000000000000], [1.00000000000000], [1.00000000000000]])



In [6]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, A.eigenvects())

egenverdi:      -0.500000000000000
egenvektor:     Matrix([[0.408248290463863], [-0.816496580927726], [0.408248290463863]])
A @ evenvektor: Matrix([[-0.204124145231932], [0.408248290463863], [-0.204124145231932]])

egenverdi:      -7.10753320192609e-65
egenvektor:     Matrix([[4.16174911291299e-65], [-0.816496580927726], [0.816496580927726]])
A @ evenvektor: Matrix([[0], [4.16174911291299e-65], [0]])

egenverdi:      1.00000000000000
egenvektor:     Matrix([[0.577350269189626], [0.577350269189626], [0.577350269189626]])
A @ evenvektor: Matrix([[0.577350269189626], [0.577350269189626], [0.577350269189626]])



In [7]:
A = sp.S("""
Matrix([
    [0, 1/2, 1/2],
    [1, 0, 0],
    [0, 1/2, 1/2]
])
""")

In [8]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

egenverdi:      -1/2
egenvektor:     Matrix([[1], [-2], [1]])
A @ evenvektor: Matrix([[-1/2], [1], [-1/2]])

egenverdi:      0
egenvektor:     Matrix([[0], [-1], [1]])
A @ evenvektor: Matrix([[0], [0], [0]])

egenverdi:      1
egenvektor:     Matrix([[1], [1], [1]])
A @ evenvektor: Matrix([[1], [1], [1]])



In [ ]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, A.eigenvects())

In [ ]:
x = sp.symbols('x')

In [ ]:
sp.solve(x**2 + 1)

In [14]:
def normer_forste_element(a):
    # Finn indeksen til det første elementet i vektoren a som ikke er null
    forste_ikke_null_element_indeks = np.argmax(a != 0)
    
    # Hent verdien til det første ikke-null elementet
    forste_ikke_null_element = a[forste_ikke_null_element_indeks]
    
    # Returner den normaliserte vektoren, der alle elementer er delt på det første ikke-null elementet
    return a / forste_ikke_null_element
    
def gauss_jordan(A):
    # Hvis matrisen kun består av nuller, returner den uendret
    if np.all(A == 0):
        return A
    
    # Hvis matrisen har én rad, normaliser den første radens første ikke-null element
    elif len(A) == 1:
        A[0] = normer_forste_element(A[0])
        return A
    else:
        # Beregn radenes summer for å normalisere matrisen 
        # (Dette er ikke nødvendig, men gjør tallene finere å regne med)
        rad_summer = np.sum(A, axis=1)
        A = A / rad_summer[:, None]  # Normaliser hver rad ved å dele med radens sum
        
        # Finn kolonner som inneholder ikke-null elementer
        ikke_null_kolonner = np.any(A != 0, axis=0)
        
        # Finn indeksen til den første kolonnen med ikke-null elementer
        forste_ikke_null_kolonne_indeks = np.argmax(ikke_null_kolonner)
        
        # Finn indeksen til raden med største verdi i den valgte kolonnen (pivot rad)
        pivot_rad_indeks = np.argmax(A[:, forste_ikke_null_kolonne_indeks])
        
        # Normaliser pivot raden
        pivot_rad = normer_forste_element(A[pivot_rad_indeks])
        
        # Bytt plass på pivot raden og den første raden
        A[pivot_rad_indeks] = A[0]
        A[0] = pivot_rad
        
        # Utfør eliminering for å gjøre alle elementene under pivoten null
        A[1:] = A[1:] - (A[1:, 0] / A[0, 0])[:, None] * A[0][None, :]
        
        # Kall gauss_jordan rekursivt på den nedre delmatrisen
        A[1:, 1:] = gauss_jordan(A[1:, 1:])
        
        # Gjør den første raden null over pivot-posisjonene til de øvrige radene.
        for rad in A[1:]:
            if np.any(rad != 0):  # Hvis raden ikke er null
                # Finn indeksen til det første ikke-null elementet i raden
                forste_ikke_null_element_indeks = np.argmax(rad != 0)
                
                # Trekk et tall gange den akutelle raden fra den første raden slik at første raden blir null over
                # pivot-elementet til den aktuelle raden
                A[0] = A[0] - (A[0, forste_ikke_null_element_indeks] / rad[forste_ikke_null_element_indeks]) * rad
    
    # Returner den resulterende matrisen
    return A


In [ ]:
A = sp.Array([
    [0, .5, .5],
    [1, 0, 0],
    [0, .5, .5]
])

In [11]:
A = np.array([
    [0, .5, .5],
    [1, 0, 0],
    [0, .5, .5]
])

In [ ]:
gauss_jordan(A)

In [16]:
def bundne_og_frie_parametre(B):
    # finner de frie parametrene i en rekkeredusert matrise
    pivot_pososjoner = set()
    for rad in B:
        if np.any(rad != 0):  # Hvis raden ikke er null
            # Finn indeksen til det første ikke-null elementet i raden
            forste_ikke_null_element_indeks = np.argmax(rad != 0)
            pivot_pososjoner.add(int(forste_ikke_null_element_indeks))
    return sorted(pivot_pososjoner), sorted(set(range(B.shape[1])).difference(pivot_pososjoner))
    

In [ ]:
bundne_og_frie_parametre(gauss_jordan(A))

In [30]:
A = np.array([
    [1, 0, 0],
    [0, 1, 1]
])

In [51]:
A = np.array([
    [0, .5, .5],
    [1, 0, 0],
    [0, .5, .5]
])

In [52]:
A

array([[0. , 0.5, 0.5],
       [1. , 0. , 0. ],
       [0. , 0.5, 0.5]])

In [53]:
def pivot_posisjoner(B):
    # finner de frie parametrene i en rekkeredusert matrise
    p_pososjoner = set()
    for idx, rad in enumerate(B):
        if np.any(rad != 0):  # Hvis raden ikke er null
            # Finn indeksen til det første ikke-null elementet i raden
            forste_ikke_null_element_indeks = np.argmax(rad != 0)
            p_pososjoner.add((idx, int(forste_ikke_null_element_indeks)))
    return list(zip(*p_pososjoner))

In [54]:
pivot_posisjoner(gauss_jordan(A))

[(1, 0), (1, 0)]

In [55]:
def bundne_og_frie_parametre(B):
    # finner de frie parametrene i en rekkeredusert matrise
    _, p_pososjoner = pivot_posisjoner(B)
    return sorted(p_pososjoner), sorted(set(range(B.shape[1])).difference(p_pososjoner))
    

In [56]:
def null_rom(A):
    res = []
    B = gauss_jordan(A)
    bparam, fparam = bundne_og_frie_parametre(B)
    for fp in fparam:
        v = np.zeros((A.shape[1], 1))
        v[fp] = 1
        for bp in bparam:
            v[bp] = -np.sum((B @ v)[bp])
        res.append(v)
    return res      

In [57]:
null_rom(A)

[array([[-0.],
        [-1.],
        [ 1.]])]

In [58]:
def partikulaer_losning(A, b=np.zeros(A.shape[1])):
    res = []
    B = gauss_jordan(np.hstack([A, b[:, None]]))
    rad_indekser, kolonne_indekser = pivot_posisjoner(B[:, :-1])
    v = np.zeros((A.shape[1], 1))
    for r, c in zip(rad_indekser, kolonne_indekser):
        v[c] = b[r]
    return v

In [59]:
b = np.array([1,1,1])

In [60]:
partikular_losning(A, b)

array([[1.],
       [1.],
       [0.]])

In [61]:
A @ b

array([1., 1., 1.])

In [ ]:
def los_system(A, b=np.zeros(A.shape[1])):
    res = []
    B = gauss_jordan(np.hstack([A, b[:, None]]))
    bparam, fparam = bundne_og_frie_parametre(B[:, :-1])
    for fp in fparam:
        v = np.zeros((A.shape[0], 1))
        v[fp] = 1
        for bp in bparam:
            v[bp] = -np.sum((B[:, :-1] @ v)[bp]) + B[bp, -1]
        res.append(v)
    return res          

In [ ]:
np.array([1,1,1])[:, None]

In [ ]:
los_system(A, np.array([0,0, 0]))

In [ ]:
los_system(A)

In [ ]:
null_rom(A)

In [ ]:
A

In [ ]:
gauss_jordan(A)

In [ ]:
bundne_og_frie_parametre(gauss_jordan(A))